In [1]:
import os
import argparse
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from utils import preprocess_data, SequenceDataset, train_model, score_model, log
from model import LSTMRegression
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.manual_seed(101)

filename = os.listdir(path='./data/')[0]
data = f'data/{filename}'

df = pd.read_csv(
    data,
    index_col='created_at'
    )
target_feature = 'Austin'
forecast_lead = 15
sequence_length = 30
learning_rate = 5e-5
hidden_units = 16
num_layers = 1
dropout=0
num_epochs = 2
batch_size = 32
num_hidden_units = 16

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
#logger = log(path="logs/",file="timeseries_training.logs")

df_train, df_test, features, target = preprocess_data(
    df,
    target_feature, 
    forecast_lead=forecast_lead
    )

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
    )

test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
    )

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
seqs, labels = next(iter(test_loader))

In [7]:
tb = SummaryWriter()
model = LSTMRegression(
    num_features=len(features), 
    hidden_units=num_hidden_units,
    num_layers=num_layers,
    dropout=dropout
    )
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
test_score = score_model(test_loader, model, loss_function)
seqs, labels = next(iter(test_loader))
tb.add_graph(model,seqs)

In [ ]:
tb = SummaryWriter()
for ix_epoch in range(num_epochs):
    print(ix_epoch)
    #logger.info(f"Epoch: {ix_epoch}")
    train_score = train_model(train_loader, model, loss_function, optimizer=optimizer)
    tb.add_scalar("Train Loss", train_score, ix_epoch)
    #logger.info(f"Train score: {train_score}")
    test_score = score_model(test_loader, model, loss_function)
    tb.add_scalar("Test Loss", test_score, ix_epoch)
    #logger.info(f"Test score: {test_score}")
tb.close()


model_name = f"LSTM_{num_hidden_units}unit_{num_layers}layer_{sequence_length}seq.pt"
torch.save(model, f"models/{model_name}")
#logger.info(f"model saved to: models/{model_name}")

In [ ]:
torch.manual_seed(101)
args = arg_parse()

filename = os.listdir(path='./data/')[0]
data = f'data/{filename}'

df = pd.read_csv(
    data,
    index_col='created_at'
    )

model = train(
    df
    )